In [1]:
!pip install bs4

   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   ------------------------ --------------- 92.2/147.9 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 1.5 MB/s eta 0:00:00


In [2]:
!pip install selenium

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.0 MB 3.3 MB/s eta 0:00:03
   - -------------------------------------- 0.3/10.0 MB 4.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/10.0 MB 4.3 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/10.0 MB 3.7 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/10.0 MB 3.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/10.0 MB 3.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.2/10.0 MB 3.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.4/10.0 MB 3.3 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/10.0 MB 3.5 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/10.0 MB 3.6 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/10.0 MB 3.6 MB/s eta 0:00:03
   -------- ------------------------------- 2.2/10.0 MB 3.6 MB/s eta 0:00:03
   ---

In [60]:
import requests
import re
from bs4 import BeautifulSoup

In [43]:
def open_website(server,name,tagline):
    opgg_url = f"https://www.leagueofgraphs.com/summoner/{server}/{name}-{tagline}"

    try:
        response = requests.get(opgg_url, headers = {'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
        if rank_element:
            rank = rank_element.text.strip()
            return rank
        else:
            print("Rank information not found for this player.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [44]:
name = "lurkz"
tagline = "h3art"
server = "euw"
leagueofgraphs = open_website(server,name,tagline)

## Get rank of user

In [45]:
leagueofgraphs


<!DOCTYPE html>

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,500,700" media="none" onload="this.media='all'" rel="stylesheet" type="text/css"/> <link href="//lolg-cdn.porofessor.gg/style.css?v=1711415117" rel="stylesheet" type="text/css">
<link href="//lolg-cdn.porofessor.gg/style.sprite.css?v=49a5d9e285a8d084807dee13e14b7ba9" rel="stylesheet" type="text/css"/>
<script src="//lolg-cdn.porofessor.gg/jquery.js?v=3"></script>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="en" http-equiv="content-language"/>
<meta content="Master - Wins: 186 (55.0%) (#11,501) / Talon: Wins: 57.3% - Played: 262 (#272) / Smolder: Wins: 60.0% - Played: 15 (#27,264) / Hwei: Wins: 57.1% - Played: 7" name="description"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="origin" name="referrer"/>
<me

### Find league tier and LP

In [123]:
rank_element = leagueofgraphs.find(class_='leagueTier')
lp_element = leagueofgraphs.find(class_='leaguePoints')

rank_helper = rank_element.text.strip()
rank = re.sub(r'\s+', ' ', rank_helper) 

lp_helper = lp_element.text.strip()
lp = int(re.findall(r'\d+', lp_helper)[0])

print(f"{name}#{tagline} is ranked as: {rank}, their LP is {lp}")


lurkz#h3art is ranked as: Master 472 LP, their LP is 472


### Find wins and losses number

In [54]:
wins_element = leagueofgraphs.find(class_='winsNumber')
losses_element = leagueofgraphs.find(class_='lossesNumber')

wins = int(wins_element.text.strip())
losses = int(losses_element.text.strip())
winrate = round((wins/(wins+losses))*100, 3)
print(f"{name}#{tagline} has: {losses} losses and {wins} wins, their winrate is {winrate}")


lurkz#h3art has: 136 losses and 161 wins, their winrate is 54.209


### Get rankings

In [108]:
rank_element = leagueofgraphs.find(class_='rank')
global_rank_element = rank_element.find(class_='highlight')
regional_rank_element = rank_element.find(class_='regionalRank')
top_rank_percentage_element = rank_element.find(class_='topRankPercentage')

top_rank_percentage_helper = str(top_rank_percentage_element.string)
global_rank_helper = str(global_rank_element.string)
regional_rank_helper = re.findall(r'\d+(?:,\d+)?', regional_rank)[0]

top_rank_percentage_number = float(re.findall(r'\d+(?:.\d+)?', top_rank_percentage)[0])
global_rank_number = re.sub(r',','', str(global_rank_helper))
regional_rank_number = re.sub(r',','', regional_rank_helper)

print(f"{name}#{tagline} is: Rank {global_rank_number} globally, making them be in the top {top_rank_percentage_number}%, their rank on {server} is {regional_rank_number} ")

lurkz#h3art is: Rank 11501 globally, making them be in the top 0.094%, their rank on euw is 1974 


### Get all played champion

In [154]:
table_element = leagueofgraphs.find('div', attrs={'data-tab-id':'championsData-soloqueue'})
table_element = leagueofgraphs.find('table')
tr_element = table_element.find_all('tr')[1:]

champions_dictionary = {}

for el in tr_element:
    champ_column = el.find('td')
    name = champ_column.find(class_='name')
    print(name)

AttributeError: 'NoneType' object has no attribute 'find'